In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import shutil

import descarteslabs as dl
from descarteslabs.catalog import Image, properties
import rasterio as rs
from rasterio.merge import merge
from tqdm import tqdm

from scripts import deploy_nn_v1
from scripts import dl_utils

In [ ]:
# User inputs

model_version = '0.0.7'
model_name = 'spectrogram_v0.0.7_2021-05-20'
model_file = '../models/' + model_name + '.h5'

roi = 'nusa_tenggara'
roi_file = f'../data/boundaries/{roi}.geojson'

# Note on dates: The date range should be longer than the spectrogram length.
# Starting on successive mosaic periods (typically: monthly), as many
# spectrograms are created as fit in the date range.
start_date = '2020-06-01'
end_date = '2021-03-01'

product_id = f'earthrise:{roi}_v{model_version}_{start_date}_{end_date}' 
product_name = product_id  # Arbitrary string - optionally set this to something more human readable.

run_local = False # If False, the model prediction tasks are async queued and sent to DL for processing.

In [ ]:
args = [
    '--roi_file',
    roi_file,
    '--product_id',
    product_id,
    '--product_name',
    product_name, 
    '--model_file',
    model_file,
    '--model_name',
    model_name,
    '--start_date',
    start_date,
    '--end_date',
    end_date,
]
if run_local:
    args.append('--run_local')

Launch Descartes job. Monitor at https://monitor.descarteslabs.com/

In [ ]:
# Because of the way DL uploads modules when queuing async tasks, we need to launch from the scripts/ folder
!cd ../scripts
!pwd

In [ ]:
deploy_nn_v1.main(args)

After the job is complete (only signaled by looking at the console), use this script to download the files. Because Descartes throws a 502 error when trying to download too many tiles. This process downloads each file individually, and then stitches them into a single geotiff

In [ ]:
# Alternatively, input a known product_id to download for an earlier model run: 
# product_id = 'earthrise:Bali_spectrogramV0.0.7_2020-06-01_2021-04-01'

In [ ]:
search = Image.search().filter(properties.product_id == product_id)
search.summary()

In [ ]:
# Select one of these available bands
product = dl.catalog.Product.get(product_id)
for b in product.bands():
    print(b.id)

In [ ]:
band = 'median'

In [ ]:
basepath = os.path.join('../data/model_outputs/heatmaps', product_id + f'mosaic-{band}')
if not os.path.exists(basepath):
    os.mkdir(basepath)

In [ ]:
image_list = [image.id for image in search]
raster_client = dl.Raster()
for image in tqdm(image_list):
    try:
        raster_client.raster(inputs = image,
                             bands = [band],
                             save=True,
                             outfile_basename = os.path.join(basepath, image),
                             srs='WGS84')
    except dl.client.exceptions.BadRequestError as e:
        print(f'Warning: {repr(e)}\nContinuing...')

In [ ]:
files_to_mosaic = []
for file in os.listdir(basepath):
    src = rs.open(os.path.join(basepath, file))
    files_to_mosaic.append(src)
mosaic, out_trans = merge(files_to_mosaic)

In [ ]:
output_metadata = src.meta.copy()

output_metadata.update({"height": mosaic.shape[1],
                        "width": mosaic.shape[2],
                        "transform": out_trans
                 }
                )
output_metadata
with rs.open(basepath + '.tif', 'w', **output_metadata) as f:
    f.write(mosaic)

In [ ]:
# Delete individual files and folder
shutil.rmtree(basepath)